# Задание 1
Разработать программу, реализующую функционал простого консольного калькулятора.
На вход текстовая строка вида «num_left operator num_right». 
num_left и num_right  - числа, со знаком
operator – допускаться простые арифметические операторы «+ - * /»  и операторы сравнения «< >  <=  >=   !=  ==».
На выходе результат выражения.


In [7]:
def calculate(expression):
    try:
        num_left, operator, num_right = expression.split()
        num_left = float(num_left)
        num_right = float(num_right)
        
        if operator == '+':
            result = num_left + num_right
        elif operator == '-':
            result = num_left - num_right
        elif operator == '*':
            result = num_left * num_right
        elif operator == '/':
            result = num_left / num_right
        elif operator == '<':
            result = num_left < num_right
        elif operator == '>':
            result = num_left > num_right
        elif operator == '<=':
            result = num_left <= num_right
        elif operator == '>=':
            result = num_left >= num_right
        elif operator == '!=':
            result = num_left != num_right
        elif operator == '==':
            result = num_left == num_right
        else:
            raise ValueError("Unknown operator")
        
        return result
    
    except ValueError as e:
        return str(e)
    except ZeroDivisionError:
        return "Division by zero error"
    
expression = input("Enter the expression (e.g., '3 + 4'): ")
print("Input:", expression)
result = calculate(expression)
print("Result:", result)

Input: 444 / 4
Result: 111.0


# Задание 2
Разработать программу, реализующую функционал консольного калькулятора.
На вход текстовая строка с математическим выражением. 
Допускаться:
несколько мат выражений разделённых символом «;»
переменные
операторы «+ - * / », оператор присваивания «=», блочный оператор «()» в т.ч. вложенные
функции «pow» «log» «sin» «sqrt» «abs».
На выходе:
Вывод списка переменных с вычисленными крайними значениями
Или сообщения об ошибке.

Применяем ООП и полиморфизм. 


In [8]:
import math
import re

class CalculatorError(Exception):
    pass

class Expression:
    def evaluate(self, variables):
        raise NotImplementedError

class Number(Expression):
    __slots__ = ('value',)
    
    def __init__(self, value):
        self.value = value
    
    def evaluate(self, variables):
        return self.value

class Variable(Expression):
    __slots__ = ('name',)
    
    def __init__(self, name):
        self.name = name
    
    def evaluate(self, variables):
        if self.name in variables:
            return variables[self.name]
        else:
            raise CalculatorError(f"Undefined variable: {self.name}")

class BinaryOperation(Expression):
    __slots__ = ('left', 'right', 'operator')
    
    def __init__(self, left, right, operator):
        self.left = left
        self.right = right
        self.operator = operator
    
    def evaluate(self, variables):
        left_value = self.left.evaluate(variables)
        right_value = self.right.evaluate(variables)
        
        if self.operator == '+':
            return left_value + right_value
        elif self.operator == '-':
            return left_value - right_value
        elif self.operator == '*':
            return left_value * right_value
        elif self.operator == '/':
            if right_value == 0:
                raise CalculatorError("Division by zero")
            return left_value / right_value
        else:
            raise CalculatorError(f"Unknown operator: {self.operator}")

class Assignment(Expression):
    __slots__ = ('variable', 'expression')
    
    def __init__(self, variable, expression):
        self.variable = variable
        self.expression = expression
    
    def evaluate(self, variables):
        result = self.expression.evaluate(variables)
        variables[self.variable.name] = result
        return result

class Function(Expression):
    __slots__ = ('function', 'argument')
    
    def __init__(self, function, argument):
        self.function = function
        self.argument = argument
    
    def evaluate(self, variables):
        arg_value = self.argument.evaluate(variables)
        
        if self.function == 'pow':
            return math.pow(arg_value, 2)
        elif self.function == 'log':
            return math.log(arg_value)
        elif self.function == 'sin':
            return math.sin(arg_value)
        elif self.function == 'sqrt':
            return math.sqrt(arg_value)
        elif self.function == 'abs':
            return abs(arg_value)
        else:
            raise CalculatorError(f"Unknown function: {self.function}")

class Calculator:
    def __init__(self):
        self.variables = {}

    def parse(self, expression):
        expression = expression.strip()
        
        if '=' in expression:
            var_name, expr = expression.split('=', 1)
            return Assignment(Variable(var_name.strip()), self.parse(expr.strip()))
        
        function_match = re.match(r'(\w+)\((.*)\)', expression)
        if function_match:
            func_name = function_match.group(1)
            arg_expr = self.parse(function_match.group(2))
            return Function(func_name, arg_expr)
        
        for operator in ['+', '-', '*', '/']:
            if operator in expression:
                left, right = expression.split(operator, 1)
                return BinaryOperation(self.parse(left.strip()), self.parse(right.strip()), operator)
        
        try:
            return Number(float(expression))
        except ValueError:
            pass
        
        return Variable(expression)

    def evaluate(self, expression):
        expr = self.parse(expression)
        result = expr.evaluate(self.variables)
        return result

    def run(self, expressions):
        results = []
        for expression in expressions.split(';'):
            try:
                result = self.evaluate(expression.strip())
                results.append(result)
            except CalculatorError as e:
                results.append(str(e))
        return results


calc = Calculator()
expressions = input("Enter the expressions (separated by ';'): ")
print("Input:", expressions)
results = calc.run(expressions)
print("Results:")
for result in results:
    print(result)
print("Variables:", calc.variables)


Input: x = 2; y = 3; z = x + y; sin(z); sqrt(4)
Results:
2.0
3.0
5.0
-0.9589242746631385
2.0
Variables: {'x': 2.0, 'y': 3.0, 'z': 5.0}
